### Training Script for CNN

The original Python script is adapted to a Jupyter Notebook to be trained and tested on Google Colab. The deployment of the model should not rely on this Notebook as it is inconsistent with the design of the system.

In [ ]:
STOCK_NAME = "MSFT" # Change this before runnign
SAVE_MODEL = False # Testing mode

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import matplotlib
# matplotlib.use('TkAgg')

import matplotlib.pyplot as plt

In [ ]:
import pandas as pd, numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

### Custom Data Loading

Since this is just built as a training trigger, we do not intend to include complicated data processing helper functions. The `get_data` function is based on a function in helper.py but has been heavily modified to **work only in this specific context**.

In [ ]:
def get_data(stock, scale=True):

    '''
    This modified version is only meant to work
    as a compromise for the CNN training notebook.
    It is designed to work well with CSV formats
    from Yahoo Finance API as well as Nasdaq
    '''

    df = pd.read_csv(f"{stock}.csv", header=0)
    df = df[1:].reset_index(drop=True) # drop the first row
    df.columns = df.columns.str.strip()

    # This parse dates in both 2024-09-11 and 09/11/2024 formats dynamically
    if df["Date"].dtype != "datetime64[ns]":
        # First, try parsing the default format (ISO 8601: YYYY-MM-DD)
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

        # If any dates failed to parse, they will be NaT, so try parsing MM/DD/YYYY format for those
        if df['Date'].isna().any():
            df['Date'] = df['Date'].fillna(pd.to_datetime(df['Date'], format='%m/%d/%Y', errors='coerce'))

    # Handle certain csv with $ symbol before number
    for col in df.columns:

        if df[col].dtype == "object":
          df[col] = df[col].replace('[\$,]', '', regex=True).astype(float)

        # If adjusted closing is available it is preferred
        # Rename Adj Close to Close to make future references simpler
        if col == "Adj Close":
          df.drop(columns=["Close"], inplace=True)
          df.rename(columns={"Adj Close": "Close"}, inplace=True)

        elif col == "Volume":
          df.drop(columns=["Volume"], inplace=True)

        elif col == "Date" and df.index.name != "Date":
          df.index.name = "Date"
          df.set_index("Date", inplace=True)

        elif col == "Close/Last":
          df.rename(columns={"Close/Last": "Close"}, inplace=True)

    df['Return'] = df["Close"].pct_change()
    df["SMA"] = df["Close"].rolling(15).mean().shift()
    df["EMA"] = df["Close"].ewm(5).mean().shift()

    R = df.Return
    if scale:
        mean = df.mean(axis=0)
        std = df.std(axis=0)
        df = (df - mean) / std
    df['Return'] = R # Return is not scaled

    # The first 15 rows are removed as they contain NaN values
    # due to the computation of technical indicators

    return df[15:]


In [ ]:
# Load the stock data
# data = pd.read_csv('AAPL_raw.csv', index_col='Date', parse_dates=True)
# data = get_data("GOOG", scale=False)

data = get_data(STOCK_NAME, scale=False)

# For this particular setup we don't use volumne nor return
labels = data["Close"].dropna()
data.drop(columns=["Return", "Close"], inplace=True)

print(data.head())
print(labels.head())

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))

close_scaler = MinMaxScaler(feature_range=(0, 1))
close_scaler.fit(labels.values.reshape(-1, 1))

data = scaler.fit_transform(data)
labels = scaler.fit_transform(labels.values.reshape(-1, 1))

# Create the training and testing datasets
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.4, train_size=0.6, shuffle=False)
test_data, val_data, test_labels, val_labels = train_test_split(test_data, test_labels, test_size=0.5, train_size=0.5, shuffle=False)

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)
print(val_data.shape)
print(val_labels.shape)

print(train_data[0])
print(train_labels[0])

In [ ]:
# Create the training and testing sequences
def create_sequences(data, labels, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 1):
        _x = data[i:(i + seq_length)]
        _y = labels[i + seq_length]
        X.append(_x)
        y.append(_y)
    return np.array(X), np.array(y)

seq_length = 10 # Two weeks of sequential data

# train_X will be a 3D array and train_y a 2D array
train_X, train_y = create_sequences(train_data, train_labels, seq_length)
test_X, test_y = create_sequences(test_data, test_labels, seq_length)
val_X, val_y = create_sequences(val_data, val_labels, seq_length)

In [ ]:
print(train_X[0])
print(train_y[0])

In [ ]:
def create_model(n_features):

    # Define the CNN-DNN model
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(seq_length, n_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))  # Increased number of units
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))  # Added another dense layer
    model.add(Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')  # Reduced learning rate

    return model

model = create_model(5)

print(model.summary())

In [ ]:
# Train the model
model.fit(train_X, train_y, epochs=75, batch_size=32, validation_data=(val_X, val_y))  # Increased epochs

# Evaluate the model
loss = model.evaluate(test_X, test_y, verbose=0)
print('Test Loss:', loss)

In [ ]:
# Make predictions
predictions = model.predict(test_X)

if SAVE_MODEL: model.save("cnn_model.keras")

print(predictions[:5])
print(test_y[:5])

# Inverse transform the predictions to get the original scale
# Use the 'Close' feature scaler to inverse transform the predictions
predictions = close_scaler.inverse_transform(predictions)
test_y = close_scaler.inverse_transform(test_y.reshape(-1, 1))

### To be improved:

Currently, you can easily notice that the x-axis does not have the most informative label... What on Earth does _"Days since 2021-01-01"_ possibly mean! And honestly, we are not even sure of the date!

The cause of the issue has been diagnosed and identified to be the `train_test_split` function since it returns an `numpy ndarray` from a `pandas df`, effectively getting rid of the `"Date"` index.

> Before the script is released to final usage and integrated with the rest of the system, the problem should be addressed for consistent evaluation metrics.

In [ ]:
plt.plot(test_y, label='Actual')
plt.xlabel('Days since test period start')
plt.ylabel('Close Price')
plt.title('Stock Price Prediction')
plt.plot(predictions, label='Predicted')
plt.legend()
plt.savefig('stock_price_prediction.png')
plt.ion()
# plt.pause(100)